In [2]:
from langchain.messages import HumanMessage,SystemMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_classic.output_parsers import StructuredOutputParser,ResponseSchema
from langchain_community.tools import tool,DuckDuckGoSearchRun

In [3]:
HISTORY = []
def set_history(conv):
    HISTORY.append(conv)

def get_history():
    return HISTORY

In [4]:
search_tool = DuckDuckGoSearchRun()

In [5]:
tools_list = [search_tool]

tools_lib = {}

for tool in tools_list:
    tools_lib[tool.name] = tool

In [6]:
prompt_plate = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
You are a helpful Agent.
Use tools when necessary to answer the user.
"""),
    MessagesPlaceholder("HISTORY"),
    HumanMessagePromptTemplate.from_template("{query}")
])


In [7]:
llm = ChatOpenAI(api_key=Key).bind_tools(tools_list)

In [8]:
query = "What is today news of bhuj"
set_history(HumanMessage(content=query))
chain = prompt_plate|llm
while True:
    response = chain.invoke({
        "query": query,
        "HISTORY": get_history()
    })

    set_history(response)

    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_result = tools_lib[tool_name].invoke(tool_call)
            set_history(tool_result)
    else:
        print("FINAL ANSWER:")
        print(response.content)
        break


FINAL ANSWER:
The latest news from Bhuj includes updates on digital arrest threats, cyber mafias targeting an elderly woman with a cyber blackmail of 83.44 lakhs in Bhuj. You can find more detailed news on this development in the news outlets like Sandesh and News18 Gujarati.
